In [1]:
import torch, transformers

tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B", cache_dir="mycache")
model = transformers.AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", cache_dir="mycache")

/home/dm/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
prefix = """"It is a fantasy role-play game.

Game Master: You are John, a wizard living in the kingdom of Larion. You have a staff and a spellbook. You finish your long journey and finally arrive at the ruin you've been looking for. You have come here searching for a mystical spellbook of great power called the book of essence. You look around and see the ancient ruins of an elf tower. The ruins have not been touched for decades. You look at the tower, and you can see a set of stone stairs that seem to lead somewhere deep inside the tower.
Player: I walk upstairs
Game Master: You climb up the stairs in the ruined tower. There is a door on the second floor of the tower, the door seems to be made of enchanted wood.
Player: I ask the door if I may to come in
Game Master: The door sighs open and you walk into the room.
Player: I take a look around
Game Master:"""

print(end=prefix)
past_key_values = None  # used to keep track of conversation history
input_dict = tokenizer([prefix], return_tensors='pt', padding=False)

output=""

with torch.inference_mode():
    for i in range(200):
        outputs = model.forward(**input_dict, use_cache=True, past_key_values=past_key_values)
        last_logits = outputs.logits[0, -1]
        
        last_logits[last_logits.topk(k=10).indices] += 10 # other logits are now e^10 times less likely to be chosen

        past_key_values = outputs.past_key_values
        token_ix = torch.multinomial(last_logits.softmax(-1), 1).item()
        prefix = tokenizer.decode([token_ix])
        output = output + tokenizer.decode([token_ix])
        if 'player' in output or 'Player' in output:
            break
        if 'Master' in output:
            break
        print(end=tokenizer.decode([token_ix]), flush=True)

        input_dict = dict(input_ids=torch.tensor([[token_ix]]))
print()

"It is a fantasy role-play game.

Game Master: You are John, a wizard living in the kingdom of Larion. You have a staff and a spellbook. You finish your long journey and finally arrive at the ruin you've been looking for. You have come here searching for a mystical spellbook of great power called the book of essence. You look around and see the ancient ruins of an elf tower. The ruins have not been touched for decades. You look at the tower, and you can see a set of stone stairs that seem to lead somewhere deep inside the tower.
Player: I walk upstairs
Game Master: You climb up the stairs in the ruined tower. There is a door on the second floor of the tower, the door seems to be made of enchanted wood.
Player: I ask the door if I may to come in
Game Master: The door sighs open and you walk into the room.
Player: I take a look around
Game Master: In the room, there are a set of stone stairs leading deeper inside the tower. You walk towards the stairs.
You climb down the stairs



In [3]:
prefix = """"It is a fantasy role-play game.

GM: You are John, a wizard living in the kingdom of Larion. You have a staff and a spellbook. You finish your long journey and finally arrive at the ruin you've been looking for. You have come here searching for a mystical spellbook of great power called the book of essence. You look around and see the ancient ruins of an elf tower. The ruins have not been touched for decades. You look at the tower, and you can see a set of stone stairs that seem to lead somewhere deep inside the tower.
Player: I walk upstairs
GM: You climb up the stairs in the ruined tower. There is a door on the second floor of the tower, the door seems to be made of enchanted wood.
Player: I ask the door if I may to come in
GM: The door sighs open and you walk into the room.
Player: I take a look around
GM:"""

print(end=prefix)
past_key_values = None  # used to keep track of conversation history
input_dict = tokenizer([prefix], return_tensors='pt', padding=False)

output=""

with torch.inference_mode():
    for i in range(200):
        outputs = model.forward(**input_dict, use_cache=True, past_key_values=past_key_values)
        last_logits = outputs.logits[0, -1]
        
        last_logits[last_logits.topk(k=10).indices] += 10 # other logits are now e^10 times less likely to be chosen

        past_key_values = outputs.past_key_values
        token_ix = torch.multinomial(last_logits.softmax(-1), 1).item()
        prefix = tokenizer.decode([token_ix])
        output = output + tokenizer.decode([token_ix])
        if 'player' in output or 'Player' in output:
            break
        if 'GM' in output:
            break
        print(end=tokenizer.decode([token_ix]), flush=True)

        input_dict = dict(input_ids=torch.tensor([[token_ix]]))
print()

"It is a fantasy role-play game.

GM: You are John, a wizard living in the kingdom of Larion. You have a staff and a spellbook. You finish your long journey and finally arrive at the ruin you've been looking for. You have come here searching for a mystical spellbook of great power called the book of essence. You look around and see the ancient ruins of an elf tower. The ruins have not been touched for decades. You look at the tower, and you can see a set of stone stairs that seem to lead somewhere deep inside the tower.
Player: I walk upstairs
GM: You climb up the stairs in the ruined tower. There is a door on the second floor of the tower, the door seems to be made of enchanted wood.
Player: I ask the door if I may to come in
GM: The door sighs open and you walk into the room.
Player: I take a look around
GM: You see a table

KeyboardInterrupt: 